## Scraping S&P 500 Components from Wikipedia 

### What we're going to do: 
1. Scrape wikipedia for lists of: 
    - Current companies in the S&P 500 
    - Historical changes to the S&P 500 
2. Write a function to generate the list of companies in the index as of a given date 
<!-- TEASER_END --> 

### Background 
Getting the current list of companies in the S&P 500 is pretty easy so we're gonna tackle that first. Reconstructing the index historically isn't so easy. Since the index is regularly rebalanced, we need a list of all the companies added and removed from the index and the date the change occurred. 

Wikipedia is nice enough to make this data available and [here's the web page](https://en.wikipedia.org/wiki/List_of_S%26P_500_companies) if you want to see what it looks like. As we'll see shortly, the format of the table of company changes is a little tricky and requires some coding gymnastics to get it into a useable format for analysis. Let's get to it. 

**Attribution:** Two of the big problems I ran into (multiple tables on a page and table data elements that span multiple rows) were solved by a fellow named Andy Roche and he was nice enough to share his approach and code in a blog post. [Here's his post](https://roche.io/2016/05/scrape-wikipedia-with-python) so be sure to check that out for a more thorough approach to wikipedia tables.

### Preliminaries 
First up, import libraries, get the site HTML with `request.get()`, then extract the tables from the BeautifulSoup ResultSet object for further cleaning. 

In [1]:
import requests
from bs4 import BeautifulSoup
import datetime 
import re 

# wikipedia page with our target tables and the initial web request 
WIKI_URL = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
req = requests.get(WIKI_URL)
req.raise_for_status()
data_retrieval_date = datetime.datetime.today() 

# here we search for all the tables on the web page and get them into a 
# beautiful soup result set  
soup = BeautifulSoup(req.content, 'lxml')
table_classes = {"class": ["sortable", "plainrowheaders"]}
wikitables = soup.findAll("table", table_classes)
type(wikitables)

bs4.element.ResultSet

### Parsing the table of current companies 
We're interested in the first two tables on the page web page. The first table is a pretty clean HTML table that lists all the companies currently in the S&P 500. We're going to traverse the table, clean thing up a bit, then store the results in a list for use later. Note the regular expression toward the end of the code block to strip out the wikipedia footnotes contained in brackets ([]). 

In [2]:
rows = wikitables[0].find_all("tr")

#parse data from table by extracting each table row ("tr" tags) 
current_companies_list = []   
for tr in rows:
    if rows.index(tr) == 0: 
        row_cells = [th.getText().strip() for th in tr.find_all('th') 
                        if th.getText().strip() != '']  
    else: 
        row_cells = (([tr.find('th').getText()] if tr.find('th') else []) 
                        + [td.getText().strip() for td in tr.find_all('td')])
    if len(row_cells) > 1: 
        # strip out brackets from reference links 
        for i, element in enumerate(row_cells): 
            if element.find('[') != -1: 
                row_cells[i] = re.sub("[\[].*?[\]]", "", element)
        current_companies_list += [row_cells]
        
current_companies_list[:2]

[['Symbol',
  'Security',
  'SEC filings',
  'GICS Sector',
  'GICS Sub Industry',
  'Headquarters Location',
  'Date first added',
  'CIK',
  'Founded'],
 ['MMM',
  '3M Company',
  'reports',
  'Industrials',
  'Industrial Conglomerates',
  'St. Paul, Minnesota',
  '',
  '0000066740',
  '1902']]

**Progress!** Now we have a clean list of lists where the first element is a list of headers and each element after it relates to a single company. We're mostly interested in the ticker symbol for each company but it doesn't hurt to keep the additional reference info for now. Not too shabby! 

### Next comes the hard part: parsing the table of changes  
Parsing data from the table of index component changes is a little more difficult. Some cells are blank, and some of the date values span multiple rows when more than one company change occurred on the same date. In order to get the data into a better format for analysis, here's what we need to do: 
- The first column is the date a change occurred so we'll write a helper function to check if it's a date
    - If we find a date, we'll hold it in a temporary variable and repeat it for each row it spans in the original HTML table  
- Next we'll clean the data so we wind up with one list element per change, accounting for the dates that span multiple rows (repeating the date when necessary) 
- We also need to explicitly keep blank cells in the added/removed columns to correctly handle the times when companies are added and none are removed or vice versa
- Each list element will be in the following format:
    - [Date, Added (ticker and company name), Removed (ticker and company name), Reason]

In [3]:
#get table of changes into bs4 result set 
row_chgs = wikitables[1].find_all("tr")

#function to check if first element is a date 
def date_check(date_text): 
    try: 
        datetime.datetime.strptime(date_text, '%B %d, %Y')
        return True 
    except ValueError: 
        return False 

# parse data as is
company_changes_list, date_holder, reason_holder = [], '', ''
for tr in row_chgs:
    if row_chgs.index(tr) == 0: 
        row_cells = [th.getText().strip() for th in tr.find_all('th') 
                        if th.getText().strip() != '']  
    else: 
        row_cells = (([tr.find('th').getText()] if tr.find('th') else []) 
                        + [td.getText().strip() for td in tr.find_all('td')])
        # check if element is a date 
        if date_check(row_cells[0]): 
            date_holder = row_cells[0]
            reason_holder = row_cells[-1]
        else: 
            row_cells.insert(0, date_holder)
            if len(row_cells) == 5: 
                row_cells.append(reason_holder) 
    if len(row_cells) > 1: 
        # strip out brackets from reference links 
        if len(row_cells) == 6: 
            row_cells[5] = re.sub("[\[].*?[\]]", "", row_cells[5])
        company_changes_list += [row_cells]

company_changes_list[:6]

[['Date', 'Added', 'Removed', 'Reason'],
 ['', 'Ticker'],
 ['January 18, 2019',
  'TFX',
  'Teleflex Inc',
  'PCG',
  'PG&E Corp',
  'PCG filing for bankrupcy'],
 ['January 2, 2019',
  'FRC',
  'First Republic Bank',
  'SCG',
  'SCANA',
  'Dominion Energy acquiring SCANA Corporation'],
 ['December 24, 2018',
  'CE',
  'Celanese Corp.',
  'ESRX',
  'Express Scripts',
  'S&P 500 constituent Cigna (NYSE: CI) acquired ESRX'],
 ['December 3, 2018',
  'LW',
  'Lamb Weston Holdings Inc',
  'COL',
  'Rockwell Collins Inc',
  'UTX acquires COL ']]

**Almost there!** Notice the second element of the list has a junk entry in it - this is due to the odd layout of the HTML table header but such is life when scraping data from the web! Here are the final housekeeping items for this list: 
- Delete the first 2 elements 
    - We mentioned the junk HTML element but the header row also has to go since we'll be sorting the list for use later in the function 
- Convert the first element of each list to a datetime object 
- Sort the entire list by the date elements in reverse order (most recent changes first) 

In [4]:
del company_changes_list[:2]
for i in company_changes_list: 
    i[0] = datetime.datetime.strptime(i[0], '%B %d, %Y') 

company_changes_list.sort(key=lambda x: x[0], reverse=True)

company_changes_list[:3]

[[datetime.datetime(2019, 1, 18, 0, 0),
  'TFX',
  'Teleflex Inc',
  'PCG',
  'PG&E Corp',
  'PCG filing for bankrupcy'],
 [datetime.datetime(2019, 1, 2, 0, 0),
  'FRC',
  'First Republic Bank',
  'SCG',
  'SCANA',
  'Dominion Energy acquiring SCANA Corporation'],
 [datetime.datetime(2018, 12, 24, 0, 0),
  'CE',
  'Celanese Corp.',
  'ESRX',
  'Express Scripts',
  'S&P 500 constituent Cigna (NYSE: CI) acquired ESRX']]

### Function that builds active company ticker lists as of a given date 
Now we'll write a function that takes a date as an argument and uses the lists we just built to return the companies that were active in the S&P 500 as of the input date. We'll also specify a function argument 'output_type' to determine the format of the returned list. A list object is easier to work with if you want do some slicing or iterating and a string is useful if you're going to feed it directly into a URL string to retrieve data from an API (more on how to do this in later posts). 

In [5]:
# input_date must be in 'YYYY-MM-DD' format 
def active_ticker_list_builder(input_date, output_type='string'):
    # convert input date to datetime object (must be in 'YYYY-MM-DD' format)
    input_date_dt = datetime.datetime.strptime(input_date, '%Y-%m-%d')
    # extract tickers only from current_companies_list (exclude the header row)
    current_ticker_list = [j[0].strip() for j in  current_companies_list[1:]]
    for i in company_changes_list:
        # if input date is more recent than first change, return current list 
        if input_date_dt > i[0]: 
            break
        elif input_date_dt <= i[0]:
            if i[1] and i[1] in current_ticker_list: 
                current_ticker_list.remove(i[1])
            if i[3]: 
                current_ticker_list.append(i[3])
        else: 
            break
    current_ticker_list.sort()
    if output_type.strip() == 'string': 
        current_tickers = ','.join(current_ticker_list)
    if output_type.strip() == 'list': 
        current_tickers = current_ticker_list 
    return current_tickers

active_tickers = active_ticker_list_builder('2017-01-01', output_type='string')

active_tickers

'A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,ADM,ADP,ADS,ADSK,AEE,AEP,AES,AET,AFL,AGN,AIG,AIV,AIZ,AJG,AKAM,ALB,ALK,ALL,ALLE,ALXN,AMAT,AME,AMG,AMGN,AMP,AMT,AMZN,AN,ANDV,ANTM,AON,APA,APC,APD,APH,APTV,ARNC,ATVI,AVB,AVGO,AVY,AWK,AXP,AYI,AZO,BA,BAC,BAX,BBBY,BBT,BBY,BCR,BDX,BEN,BF-B,BHGE,BIIB,BK,BKNG,BLK,BLL,BMY,BRK-B,BSX,BWA,BXP,C,CA,CAG,CAH,CAT,CB,CBRE,CBS,CCI,CCL,CELG,CERN,CF,CFG,CHD,CHK,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA,CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COL,COO,COP,COST,COTY,CPB,CPRI,CRM,CSCO,CSRA,CSX,CTAS,CTL,CTSH,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DLR,DLTR,DNB,DOV,DPS,DRI,DTE,DUK,DVA,DVN,DWDP,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,ENDP,EOG,EQIX,EQR,EQT,ES,ESRX,ESS,ETFC,ETN,ETR,EVHC,EW,EXC,EXPD,EXPE,EXR,F,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FL,FLIR,FLR,FLS,FMC,FOX,FOXA,FRT,FSLR,FTI,FTR,FTV,GD,GE,GGP,GILD,GIS,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GT,GWW,HAL,HAR,HAS,HBAN,HBI,HCA,HCP,HD,HES,HIG,HOG,HOLX,HON,HP,HPE,HPQ,HRB,HRL,HRS,HSIC,HST,HSY,HUM,IBM,ICE,IFF,ILMN

### Closing thoughts  
This is obviously a pretty specific use case, so hopefully you find some parts of it helpful. I wrote this as a stand-alone example but if this something you're doing regularly, it's probably better to store these tables in a database and run a daily script with a chron job or something to update the changes (that's my plan, at least). 

If you're only interested in the current list of companies, the first table usually behaves pretty well with the Pandas `read_html()` function, so that's an easy workaround if you don't care about all the historical changes. 

Drop me a line if you see any bugs/errors or if you have a better way to do this. 